In [ ]:
# 读取上传的公司为行格式文件
df = pd.read_excel("D:/HuaweiMoveData/Users/86136/Desktop/data/ESG_环境类_四项指标_公司为行_横向格式.xlsx")

# 保存公司列名
company_col = "公司"

# 对每个指标列进行分位数转换，缺失值使用该列均值填充后再计算分位数
quantile_scaled = df.copy()
for col in df.columns:
    if col != company_col:
        col_values = df[col]
        filled = col_values.fillna(col_values.mean())
        quantiles = filled.rank(pct=True)
        quantile_scaled[col] = quantiles

# 导出处理后的分位数评分表
quantile_path = "D:/HuaweiMoveData/Users/86136/Desktop/data/ESG_环境类_四项指标_分位数替换版.xlsx"
quantile_scaled.to_excel(quantile_path, index=False)

In [ ]:
# 读取刚刚处理过的分位数表
df_quantile = pd.read_excel("D:/HuaweiMoveData/Users/86136/Desktop/data/ESG_环境类_四项指标_分位数替换版.xlsx")

# 公司列不变，其他指标乘以10
df_scaled = df_quantile.copy()
for col in df_scaled.columns:
    if col != "公司":
        df_scaled[col] = df_scaled[col] * 10

# 导出结果（0-10 评分）
score_path = "D:/HuaweiMoveData/Users/86136/Desktop/data/ESG_环境类_四项指标_0到10得分表.xlsx"
df_scaled.to_excel(score_path, index=False)


In [ ]:
import numpy as np

# 读取0-10得分表
df_score = pd.read_excel("D:/HuaweiMoveData/Users/86136/Desktop/data/ESG_环境类_四项指标_0到10得分表.xlsx")

# 权重
weight_exposure = 0.329  # for Absolute emissions
weight_management = 0.263  # for Emission intensities

# 获取对应得分列
exp_col = "Absolute emissions by scope (Scope 1, Scope 2, Scope 3 if applicable)"
mgmt_col = "Emission intensities by scope"

# 加权得分计算
df_score["加权后_敞口"] = df_score[exp_col] * weight_exposure
df_score["加权后_管理"] = df_score[mgmt_col] * weight_management

# 反推真实得分（按照相对权重还原）
total_weight = weight_exposure + weight_management
df_score["敞口得分"] = df_score["加权后_敞口"] / weight_exposure
df_score["管理得分"] = df_score["加权后_管理"] / weight_management

# 应用关键议题得分公式：7 - (max(敞口得分, 2) - 管理得分)
df_score["关键议题得分"] = 7 - (np.maximum(df_score["敞口得分"], 2) - df_score["管理得分"])
df_score["关键议题得分"] = df_score["关键议题得分"].clip(lower=0, upper=10).round(1)

# 添加为第六列
df_score.insert(5, "关键议题得分", df_score.pop("关键议题得分"))

# 导出
issue_score_path = "D:/HuaweiMoveData/Users/86136/Desktop/data/ESG_环境类_含关键议题得分表.xlsx"
df_score.to_excel(issue_score_path, index=False)


In [ ]:
# 水议题的权重
weight_exposure_water = 0.211  # for Total water consumption
weight_management_water = 0.197  # for Water consumption intensity

# 获取得分列
exp_col_water = "Total water consumption"
mgmt_col_water = "Water consumption intensity"

# 加权得分
df_score["加权后_敞口（水）"] = df_score[exp_col_water] * weight_exposure_water
df_score["加权后_管理（水）"] = df_score[mgmt_col_water] * weight_management_water

# 反推真实得分
df_score["敞口得分（水）"] = df_score["加权后_敞口（水）"] / weight_exposure_water
df_score["管理得分（水）"] = df_score["加权后_管理（水）"] / weight_management_water

# 应用议题得分公式：7 - (max(敞口, 2) - 管理)
df_score["关键议题得分（水）"] = 7 - (np.maximum(df_score["敞口得分（水）"], 2) - df_score["管理得分（水）"])
df_score["关键议题得分（水）"] = df_score["关键议题得分（水）"].clip(lower=0, upper=10).round(1)

# 插入在最后一列
df_score["关键议题得分（水）"] = df_score.pop("关键议题得分（水）")

# 导出更新表格
water_issue_score_path = "D:/HuaweiMoveData/Users/86136/Desktop/data/ESG_环境类_含两个关键议题得分表.xlsx"
df_score.to_excel(water_issue_score_path, index=False)


In [ ]:
# 定义两个关键议题的权重
weight_1 = 0.592  # Absolute emissions + Emission intensity
weight_2 = 0.408  # Water consumption + Water consumption intensity

# 组合加权总得分列
df_score["议题总得分"] = (
    df_score["关键议题得分"] * weight_1 +
    df_score["关键议题得分（水）"] * weight_2
).round(2)

# 导出添加“议题总得分”的最终表格
final_issue_score_path = "D:/HuaweiMoveData/Users/86136/Desktop/data/ESG_环境类_含总议题得分表.xlsx"
df_score.to_excel(final_issue_score_path, index=False)